# SCP Toolbox Workshop

___A tutorial on generating dynamically feasible trajectories reliably and efficiently___

Monday, February 7, 2022

Rocky Mountain AAS GN&C Conference, Breckenridge, CO

<h2 style="color: black;">
    <span style="background-color: #ECBE7B; padding: 1px;">&ensp;Part 4: Rocket-Landing Guidance&ensp;</span>
</h2>

In [151]:
import Pkg
Pkg.activate("..")

## these lines are required for local installations
# Pkg.develop(path="../../SCPToolbox.jl/")
# Pkg.precompile()

using SCPToolbox
using PyCall, PyPlot, Colors, LinearAlgebra

# Import the different possible low-level convex solvers
using ECOS
# using Ipopt
# using SCS
# using OSQP

  Activating project at `~/Downloads/tmp/SCPToolbox_tutorial`


In this part of the tutorial, you will participate in formulating and solving a Lunar rocket landing trajectory optimization problem.

The rocket has a 2-dimensional position and a rotation, and a gimbaled rocket engine. The goal is to take the rocket from an airborne state to a soft touchdown on the ground.

<center>
    <img src="media/p4-rocket-landing-overview.png"
         alt="rocket-landing"
         style="background-color: white; width: 300px; display: block; margin-left: auto; margin-right: auto;"/>
    <br />
    <b>Figure.</b> Illustration of the rocket soft landing problem.
    <br />
    <br />
</center>

Begin by defining a new `TrajectoryProblem` object.

In [152]:
pbm =TrajectoryProblem(); # Enter your code here

<details>

<summary>Answer</summary>



```julia
pbm = TrajectoryProblem();
```
</details>

## Rocket parameters

Any trajectory optimization problem begins by defining the constant parameters of the problem. For example, how much does the rocket weigh? How high up does it start?

We define these below.

In [153]:
# Environment parameters
g = 1.625 # [m/s^2] Gravitational acceleration of the planet
g_E = 9.807; # [m/s^2] Earth gravity

In [154]:
# Mechanical parameters
m_wet = 25e3 # [kg] Initial mass
L = 0.5 # [m] Thrust lever arm
J = 100e3; # [kg*m^2] Moment of inertia

In [155]:
# Propulsion parameters
Isp = 370 # [s] Specific impulse
T_min = 20e3 # [N] Minimum thrust
T_max = 80e3 # [N] Maximum thrust
α = 1 / (Isp * g_E) # [kg/s/N] Fuel consumption rate coefficient
δ_max = deg2rad(10.0); # [rad] Maximum gimbal angle

In [156]:
# Trajectory parameters
γ_gs = deg2rad(45.0); # [rad] Glideslope angle (measured from the horizon)

## Rocket dynamics

The partial rocket dynamics are given by the following ordinary differential equations:

\begin{align}
\dot x &= v_x, \\
\dot y &= \mathrm{\color{grey}{TODO}}, \\
\dot v_x &= -\frac{T}{m}\sin(\theta+\delta), \\
\dot v_y &= \mathrm{\color{grey}{TODO}}, \\
\dot \theta &= \mathrm{\color{grey}{TODO}}, \\
\dot \omega &= -\frac{LT}{J}\sin(\delta), \\
\dot m &= -\alpha T.
\end{align}

<details>

<summary>Answer</summary>

\begin{align}
    \dot y &= v_y, \\
    \dot v_y &= \frac{T}{m}\cos(\theta+\delta)-g, \\
    \dot \theta &= \omega.
\end{align}

</details>

Define the state vector $x\in\mathbb R^n$, with the elements in the same order as they appear on the left-hand side of the ordinary differential equations above:

$$
x = \begin{bmatrix}
\mathrm{\color{grey}{TODO}}
\end{bmatrix}.
$$

<details>

<summary>Answer</summary>

$$
x = \begin{bmatrix}
x \\ y \\ v_x \\ v_y \\ \theta \\ \omega \\ m
\end{bmatrix}
$$

</details>

Define the input vector $u\in\mathbb R^m$, with the thrust coming first:

$$
u = \begin{bmatrix}
\mathrm{\color{grey}{TODO}}
\end{bmatrix}.
$$

<details>

<summary>Answer</summary>

$$
u = \begin{bmatrix}
T \\ \delta
\end{bmatrix}
$$

</details>

Complete the function below that evaluates the right-hand side of the dynamics ordinary differential equations above. The function signature is:

$$
\dot x = f_{rocket}(x, u),
$$

where the arguments are the state and input and the output is the state time derivative.

In [157]:


f_rocket(x, u) = begin
    rx, ry, vx, vy, θ, ω, m = x
    T, δ = u
    # Enter your code here
    return [vx; vy; -(T/m)*sin(θ+δ); (T/m)*cos(θ+δ)-g; ω; -(L*T/J)*sin(δ); -α*T]
end;



<details>

<summary>Answer</summary>

```julia
f_rocket(x, u) = begin
    rx, ry, vx, vy, θ, ω, m = x
    T, δ = u
    return [vx; vy; -(T/m)*sin(θ+δ); (T/m)*cos(θ+δ)-g; ω; -(L*T/J)*sin(δ); -α*T]
end;
```

</details>

Remember from the Dubin's car example that SCP Toolbox operates on a normalized-time problem where time takes values in $[0,1]$.

We are going to solve a free-final time rocket landing problem, and this requires us to introduce a one-element parameter vector that holds the time dilation coefficient:

$$
p = \begin{bmatrix}
t_f
\end{bmatrix}.
$$

The normalized-time dynamics are given by:

$$
\dot x = t_f f_{rocket}(x, u) \equiv f(x, u, p).
$$

This allows us to write the normalized-time dynamics in Julia as follows.

In [158]:
f(x, u, p) = begin
    tf, = p
    f_rocket(x, u)*tf
end;

Based on the above definitions of the state ($x$), the input ($u$), and the parameter ($p$), record their dimensions ($n$, $m$, and $d$, respectively).

In [159]:
n = 7# Enter your code here
m =2 # Enter your code here
d = 1# Enter your code here

1

<details>

<summary>Answer</summary>

```julia
n = 7
m = 2
d = 1
```

</details>

Use the SCP Toolbox API to define the state, input, and parameter dimensions in the problem.

In [160]:
problem_set_dims!(pbm, n, m, d) # Enter your code here

<details>

<summary>Answer</summary>

```julia
problem_set_dims!(pbm, n, m, d)
```

</details>

### Jacobians

We need to define the Jacobians of the dynamics with respect to the state, input, and parameter vectors.

Remember the definition of the Jacobian of a general function $g(z):\mathbb R^n\to\mathbb R^m$:

$$
\nabla_z g(z) = \begin{bmatrix}
\frac{\partial g(z)}{\partial z_1} &
\frac{\partial g(z)}{\partial z_2} &
\cdots &
\frac{\partial g(z)}{\partial z_n}
\end{bmatrix}\in\mathbb R^{m\times n}.
$$

With the above in mind, write down "in math" the Jacobian of the dynamics with respect to $x$:

$$
A(x, u, p) = \nabla_x f(x, u, p) = \begin{bmatrix}
\mathrm{\color{grey}{TODO}}
\end{bmatrix}\in\mathbb R^{n\times n}.
$$

To help, here is what the column corresponding to the state $x_5\equiv\theta$ looks like:

$$
\begin{bmatrix}
0\\0\\- \frac{T t_f \cos{\left(\delta + \theta \right)}}{m}\\- \frac{T t_f \sin{\left(\delta + \theta \right)}}{m}\\0\\0\\0\end{bmatrix}.
$$

<details>

<summary>Answer</summary>

$$
A(x, u, p) = 
\begin{bmatrix}
0 & 0 & t_f & 0 & 0 & 0 & 0\\
0 & 0 & 0 & t_f & 0 & 0 & 0\\
0 & 0 & 0 & 0 & - \frac{T t_f \cos{\left(\delta + \theta \right)}}{m} & 0 & \frac{T t_f \sin{\left(\delta + \theta \right)}}{m^{2}}\\0 & 0 & 0 & 0 & - \frac{T t_f \sin{\left(\delta + \theta \right)}}{m} & 0 & - \frac{T t_f \cos{\left(\delta + \theta \right)}}{m^{2}}\\0 & 0 & 0 & 0 & 0 & t_f & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0\end{bmatrix}.
$$

</details>

Now implement the function in Julia.

In [161]:


A(x, u, p) = begin
    rx, ry, vx, vy, θ, ω, m = x
    T, δ = u
    tf, = p
    # Enter your code here
    return [0 0 1 0       0       0       0
            0 0 0 1       0       0       0
            0 0 0 0 -T/m*cos(θ+δ) 0 T/m^2*sin(θ+δ)
            0 0 0 0 -T/m*sin(θ+δ) 0 -T/m^2*cos(θ+δ)
            0 0 0 0       0       1       0
            0 0 0 0       0       0       0
            0 0 0 0       0       0       0]*tf
end;



<details>

<summary>Answer</summary>

```julia
A(x, u, p) = begin
    rx, ry, vx, vy, θ, ω, m = x
    T, δ = u
    tf, = p
    return [0 0 1 0       0       0       0
            0 0 0 1       0       0       0
            0 0 0 0 -T/m*cos(θ+δ) 0 T/m^2*sin(θ+δ)
            0 0 0 0 -T/m*sin(θ+δ) 0 -T/m^2*cos(θ+δ)
            0 0 0 0       0       1       0
            0 0 0 0       0       0       0
            0 0 0 0       0       0       0]*tf
end
```

</details>

Similarly, write down the Jacobian with respect to the input, $u$:

$$
B(x, u, p) = \nabla_u f(x, u, p) = \begin{bmatrix}
\mathrm{\color{grey}{TODO}}
\end{bmatrix}\in\mathbb R^{n\times m}.
$$

<details>

<summary>Answer</summary>

$$
B(x, u, p) = 
\begin{bmatrix}
0 & 0\\0 & 0\\- \frac{t_f \sin{\left(\delta + \theta \right)}}{m} & - \frac{T t_f \cos{\left(\delta + \theta \right)}}{m}\\\frac{t_f \cos{\left(\delta + \theta \right)}}{m} & - \frac{T t_f \sin{\left(\delta + \theta \right)}}{m}\\0 & 0\\- \frac{L t_f \sin{\left(\delta \right)}}{J} & - \frac{L T t_f \cos{\left(\delta \right)}}{J}\\- \alpha t_f & 0
\end{bmatrix}.
$$

</details>

Now implement the function in Julia.

In [162]:


B(x, u, p) = begin
    rx, ry, vx, vy, θ, ω, m = x
    T, δ = u
    tf, = p
    # Enter your code here
    return [     0            0
                 0            0
            -sin(θ+δ)/m -T/m*cos(θ+δ)
             cos(θ+δ)/m -T/m*sin(θ+δ)
                 0            0
            -L/J*sin(δ) -L*T/J*cos(δ)
                -α            0       ]*tf
end;



<details>

<summary>Answer</summary>

```julia
B(x, u, p) = begin
    rx, ry, vx, vy, θ, ω, m = x
    T, δ = u
    tf, = p
    return [     0            0
                 0            0
            -sin(θ+δ)/m -T/m*cos(θ+δ)
             cos(θ+δ)/m -T/m*sin(θ+δ)
                 0            0
            -L/J*sin(δ) -L*T/J*cos(δ)
                -α            0       ]*tf
end
```

</details>

Finally, write down the Jacobian with respect to the parameter vector, $p$:

$$
F(x, u, p) = \nabla_p f(x, u, p) = \begin{bmatrix}
\mathrm{\color{grey}{TODO}}
\end{bmatrix}\in\mathbb R^{n\times d}.
$$

<details>

<summary>Answer</summary>

$$
F(x, u, p) = 
f_{rocket}(x, u).
$$

</details>

Now implement the function in Julia.

In [163]:


F(x, u, p) = begin
    rx, ry, vx, vy, θ, ω, m = x
    T, δ = u
    tf, = p
    # Enter your code here
    return reshape(f(x, u, p)/tf, 7, 1)
end;



<details>

<summary>Answer</summary>

```julia
F(x, u, p) = begin
    rx, ry, vx, vy, θ, ω, m = x
    T, δ = u
    tf, = p
    return reshape(f(x, u, p)/tf, 7, 1)
end;
```

</details>

The SCP Toolbox API also passes other arguments to the dynamics and its Jacobians:
* `t`: the time;
* `k`: the discrete time index;
* `pbm`: the `TrajectoryProblem` object that we defined at the very beginning.

We write a simple wrapper to satisfy the API.

In [164]:
wrap(func) = (t, k, x, u, p, pbm) -> func(x, u, p);
f_scp = wrap(f)
A_scp = wrap(A)
B_scp = wrap(B)
F_scp = wrap(F)

#110 (generic function with 1 method)

Use the above functions `f_scp`, `A_scp`, `B_scp`, `F_scp` in order to define the dynamics using the SCP Toolbox API.

In [165]:
problem_set_dynamics!(pbm, f_scp, A_scp, B_scp, F_scp)

<details>

<summary>Answer</summary>

```julia
problem_set_dynamics!(pbm, f_scp, A_scp, B_scp, F_scp)
```

</details>

## Boundary Conditions

Let's define the boundary conditions on the state as a fixed "airborne" point above the landing site at the start, and a point at the landing state at the end of the trajectory. Because the rocket depletes mass, and we do not know by how much ahead of time, the final mass is left unspecified.

In [166]:
km2m(x) = x*1e3 # Conversion from km to m
kph2mps(v) = v/3.6 # Conversion from km/h to m/s

# Initial state
x_0 = [km2m(0.5)
       km2m(1.4)
       kph2mps(80)
       kph2mps(-100)
       deg2rad(-30.0)
       deg2rad(0.0)
       m_wet]

# Terminal state (without mass)
x_f = zeros(6);

Recall that the boundary conditions are defined as the equality constraints:

\begin{align}
g_{ic}(x(0), p) &= 0, \\
g_{tc}(x(1), p) &= 0.
\end{align}

Given the definitions of $x_0$ (`x_0`) and $x_f$ (`x_f`), define the $g_{ic}$ and $g_{tc}$ functions.

\begin{align}
g_{ic}(x, p) &= \mathrm{\color{grey}{TODO}}, \\
g_{tc}(x, p) &= \mathrm{\color{grey}{TODO}}.
\end{align}

<details>

<summary>Answer</summary>

\begin{align}
g_{ic}(x, p) &= x-x_0, \\
g_{tc}(x, p) &= E_{m} x-x_f,
\end{align}
where:
$$
E_{m} = \begin{bmatrix}
I\in\mathbb R^{6\times 6} &\mid & 0
\end{bmatrix}\in\mathbb R^{6\times 7}.
$$

</details>

Now implement these functions in Julia.

In [167]:


g_ic(x, p) = x-x_0 # Enter your code here
g_tc(x, p) = x[1:6]-x_f; # Enter your code here



<details>

<summary>Answer</summary>

```julia
g_ic(x, p) = x-x_0
g_tc(x, p) = x[1:6]-x_f;
```

</details>

Again, we need to provide the Jacobians of $g_{ic}$ and $g_{tc}$ for the SCP algorithm to work with. This is needed because, in general, these may be nonaffine functions. Define the following Jacobians:

\begin{align}
H_0(x(0), p) &= \nabla_x g_{ic}(x(0), p), \\
K_0(x(0), p) &= \nabla_p g_{ic}(x(0), p), \\
H_f(x(1), p) &= \nabla_x g_{tc}(x(1), p), \\
K_f(x(1), p) &= \nabla_p g_{tc}(x(1), p).
\end{align}

When a Jacobian is not provided, the SCP Toolbox assumes that it is zero. Write down the non-zero Jacobians for $g_{ic}$ and $g_{tc}$ (omit those that are zero).

\begin{align}
H_0(x(0), p) &= \mathrm{\color{grey}{TODO}}, \\
K_0(x(0), p) &= \mathrm{\color{grey}{TODO}}, \\
H_f(x(1), p) &= \mathrm{\color{grey}{TODO}}, \\
K_f(x(1), p) &= \mathrm{\color{grey}{TODO}}.
\end{align}

<details>

<summary>Answer</summary>
\begin{align}
H_0(x(0), p) &= I\in\mathbb R^{7\times 7}, \\
H_f(x(1), p) &= \begin{bmatrix} I\in\mathbb R^{6\times 6} & 0\end{bmatrix}\in\mathbb R^{6\times 7}.
\end{align}

</details>

Now implement the non-zero Jacobians in Julia. You can delete the lines for the zero Jacobians.

In [168]:


H_0(x, p) = I(7) # Enter your code here
H_f(x, p) = collect(hcat(I(6), zeros(6))); # Enter your code here



<details>

<summary>Answer</summary>

```julia
H_0(x, p) = I(7)
H_f(x, p) = collect(hcat(I(6), zeros(6)));
```

</details>

We now use the SCP Toolbox API in order to associate the boundary conditions with the trajectory optimization problem. The code below configures the above functions to accept the arguments that the toolbox will provide them with. The API function `problem_set_bc!` is then used to link the boundary conditions to the trajectory problem.

Your job is to uncomment the `problem_set_bc!` calls, and remove the zero Jacobians from each call.

In [169]:


wrap(func) = (x, p, pbm) -> func(x, p)
# problem_set_bc!(pbm, :ic, wrap(g_ic), wrap(H_0), wrap(K_0))
# problem_set_bc!(pbm, :tc, wrap(g_tc), wrap(H_f), wrap(K_f));
problem_set_bc!(pbm, :ic, wrap(g_ic), wrap(H_0))
problem_set_bc!(pbm, :tc, wrap(g_tc), wrap(H_f));



<details>

<summary>Answer</summary>

```julia
problem_set_bc!(pbm, :ic, wrap(g_ic), wrap(H_0))
problem_set_bc!(pbm, :tc, wrap(g_tc), wrap(H_f));
```

</details>

## State Constraints

In order to not collide with the ground or any obstacles (such as hills) surrounding the landing site, the rocket's position is constrained to be above a certain glideslope, as illustrated in the figure at the top of this notebook.

Write down the glideslope constraint on the position vector $[x;y]$. (Hint: it is a convex inequality and involves $|x|$).

$$
\mathrm{\color{grey}{TODO}}\le 0.
$$

<details>

<summary>Answer</summary>

$$
|x|-\frac{y}{\tan(\gamma_{gs})}\le 0.
$$

</details>

What kind of convex cone can represent this constraint?

Enter your answer in term of the Julia variable that is used to by the SCP Toolbox to denote the cone (recall the table from the Part 1 notebook): `TODO`.

<details>

<summary>Answer</summary>

`L1`

</details>

Now use the SCP Toolbox's `@add_constraint` macro (which you learned about in Part 1) in order to define the glideslope constraint. Assume that the function receives `x` (the full state vector) and `clp` (the `ConicProgram` object).

In [170]:
glideslope(x, clp) = @add_constraint(clp, L1, x -> begin
        x, y = x[1], x[2]
        return [y/tan(γ_gs); x]
    end);

<details>

<summary>Answer</summary>

```julia
glideslope(x, clp) = @add_constraint(clp, L1, x -> begin
        x, y = x[1], x[2]
        return [y/tan(γ_gs); x]
    end);
```
    
</details>

Let's associate this constraint with the trajectory problem using the SCP Toolbox API for convex state constraints.

In [171]:
problem_set_X!(pbm, (t, k, x, p, pbm, clp) -> glideslope(x, clp))

## Control Constraints

The control constraints ensure that the rocket's propulsion system can actually reproduce the inputs that are computed by the optimal trajectory.

For this problem, we will constraint the thrust to $T\in [T_{\min}, T_{\max}]$ and the gimbal angle $\delta\in [-\delta_{\max},\delta_{\max}]$.

Write these control constraints as cones. Write the constraints separately for thrust lower bound, upper bound, and gimbal angle (thus you should end up with 3 cones). Only use the cones that are supported by the SCP Toolbox (refer to Part 1).

<details>

<summary>Answer</summary>

\begin{align*}
T_{\min}-T &\in \mathbb K_{\le 0}, \\
T-T_{\max} &\in \mathbb K_{\le 0}, \\
\begin{bmatrix}
\delta_{\max} \\ \delta
\end{bmatrix} &\in \mathbb K_{1}.
\end{align*}
    
</details>

Now implement each constraint in Julia using `@add_constraint`.

In [172]:


# Enter your code here
thrust_upper_bound(T, clp) = @add_constraint(clp, NONPOS, T -> T-T_max)
thrust_lower_bound(T, clp) = @add_constraint(clp, NONPOS, T -> T_min-T)
gimbal_limit(δ, clp) = @add_constraint(clp, L1, δ -> vcat(δ_max, δ));



<details>

<summary>Answer</summary>

```julia
thrust_upper_bound(T, clp) = @add_constraint(clp, NONPOS, T -> T-T_max)
thrust_lower_bound(T, clp) = @add_constraint(clp, NONPOS, T -> T_min-T)
gimbal_limit(δ, clp) = @add_constraint(clp, L1, δ -> vcat(δ_max, δ));
```
    
</details>

Now call the SCP Toolbox API function `problem_set_U!` in order to associate the convex input constraints with the trajectory problem.

In [173]:


problem_set_U!(pbm, (t, k, u, p, pbm, clp) -> begin
        # Enter your code here
        T, δ = u[1], u[2]
        thrust_upper_bound(T, clp)
        thrust_lower_bound(T, clp)
        gimbal_limit(δ, clp)
    end)



<details>

<summary>Answer</summary>

```julia
problem_set_U!(pbm, (t, k, u, p, pbm, clp) -> begin
        T, δ = u[1], u[2]
        thrust_upper_bound(T, clp)
        thrust_lower_bound(T, clp)
        gimbal_limit(δ, clp)
    end)
```
    
</details>

## Objective Function

A common objective for rocket landing guidance is to compute a trajectory that uses the least amount of fuel. This results in a lighter vehicle design that can carry more useful scientific payload, and other beneficial tradeoffs.

Minimizing fuel use is equivalent to maxizing the final mass of the rocket (since the least amount of fuel will have then been depleted). The SCP Toolbox seeks to __minimize__ the following objective function:

$$
J(x, u, p) = \phi(x(1), p) + \int_0^1 \Gamma(x(t), u(t), p) dt.
$$

Define $\phi$ and $\Gamma$ for maximizing the final mass:

\begin{align*}
\phi(x(1), p) &= \mathrm{\color{grey}{TODO}}, \\
\Gamma(x(t), u(t), p) &= \mathrm{\color{grey}{TODO}}.
\end{align*}

Be sure to _normalize_ the functions, meaning that you should multiply them by a constant coefficient such that along any reasonable trajectory the following (approximately) hold: $\phi(\cdot)\in [0,1]$ and $\Gamma(\cdot)\in [0,1]$.

(Hint: one of the functions may have to be zero!)

<details>

<summary>Answer</summary>

\begin{align*}
\phi(x(1), p) &= -\frac{ m(1) }{ m_{wet} }, \\
\Gamma(x(t), u(t), p) &= 0.
\end{align*}
    
</details>

Now implement the functions in Julia. (Hint: you can delete any function that is zero).

In [174]:


ϕ(x, p) = -x[end]/m_wet;
#Γ(x, u, p) = # Enter your code here

ϕ (generic function with 1 method)

<details>

<summary>Answer</summary>

```julia
ϕ(x, p) = -x[end]/m_wet;
```
    
</details>

We now have to pass the above functions to the objective function definition of the trajectory problem. The SCP Toolbox allows us to do this via the API functions `problem_set_terminal_cost!` and `problem_set_running_cost!`. If either one is missing, you can omit the call and it will be assumed that the corresponding part of the objective is zero.

In the cell below, uncomment the lines corresponding to the non-zero components of the cost.

In [175]:
problem_set_terminal_cost!(pbm, (x, p, pbm) -> ϕ(x, p))
# problem_set_running_cost!(pbm, (t, k, x, u, p, pbm) -> Γ(x, u, p))

<details>

<summary>Answer</summary>

```julia
problem_set_terminal_cost!(pbm, (x, p, pbm) -> ϕ(x, p));
```
    
</details>

## Initial Trajectory Guess

The initial trajectory guess is required to get an SCP algorithm started. This guess can be quite coarse, and does not need to be feasible with respect to the dynamics and nonconvex constraints. Some SCP algorithms require the guess to be feasible with respect to the convex constraints, which is usually easy to do (in fact, it can be done automatically via a convex projection).

A straight-line guess is sufficient for the rocket landing problem. The following code interpolates from the initial to the final condition. The duration is set to the time it would take an average thrust to slow the rocket down to a hover. The angular rate is set to a constant value that rotates the rocket from its initial tilt to the final upright orientation.

In [176]:
T_guess = T_min+0.5*(T_max-T_min) # [N] Initial thrust guess
v_0, v_f = x_0[3:4], x_f[3:4]
tf_guess = norm(v_f-v_0, 2)/(T_guess/m_wet-g)
Δm_guess = -α*T_guess*tf_guess
m_dry_guess = x_0[end]+Δm_guess

state_guess(N) = begin
    x_guess = straightline_interpolate(x_0, vcat(x_f, m_dry_guess), N)
    x_guess[6, :] .= (x_f[5]-x_0[5])/tf_guess
    return x_guess
end;

Let's guess the input to be a constant thrust of `T_guess` amount, and zero gimbal angle. Complete the following function using `straightline_interpolate`.

In [177]:
input_guess(N) = straightline_interpolate([T_guess; 0.0], [T_guess; 0.0], N); # Enter your code here

<details>

<summary>Answer</summary>

```julia
input_guess(N) = straightline_interpolate([T_guess; 0.0], [T_guess; 0.0], N);
```
    
</details>

The following call the `problem_set_guess!` lets the SCP Toolbox know to use these initial guesses for the SCP solution.

In [178]:


problem_set_guess!(pbm, (N, pbm) -> begin
    x = state_guess(N)
    u = input_guess(N)
    p = [tf_guess]
    return x, u, p
end)



## Variable Scaling

Variable scaling is important when the solution variables are of widely different magnitudes. This is definitely the case for the rocket landing problem. For example, the gimbal angle varies in the $[-\delta_{\max}, \delta_{\max}]$ (which is on the order of $0.1$) while thrust can be as high as 100 kilonewtons and mass can be as high as several thousan kilograms.

The SCP Toolbox performs automatic scaling for variables whose magnitudes are naturally constrained by the convex constraints. However, you can also set the variable scaling manually by providing the expected value ranges for each variable. This is done using the SCP Toolbox API function `problem_advise_scale!!`.

The function accepts the element index in the state, input, or parameter vector, and a tuple `(min, max)` range for the expected values of the corresponding variable. The bounds don't have to be exact, but a well-chosen set of tight bounds will do better.

In the cell directly below, define the ranges (as `(min, max)` tuples) for the mass and thrust.

In [179]:


mass_range = (m_wet+Δm_guess, m_wet) # Enter your code here
thrust_range = (T_min, T_max) # Enter your code here



(20000.0, 80000.0)

<details>

<summary>Answer</summary>

```julia
mass_range = (m_wet+Δm_guess, m_wet)
thrust_range = (T_min, T_max)
```
    
</details>

In [180]:
rx_range_box = max(x_0[1], 1.0)
ry_range_box = max(x_0[2], 1.0)
vx_range_box = max(abs(x_0[3]), 1.0)
vy_range_box = min(x_0[4], -1.0)
θ_range_box = deg2rad(20.0)
ω_range_box = max(abs(x_0[6]), deg2rad(5.0))
δ_range_box = deg2rad(1.0)

problem_advise_scale!(pbm, :state, 1, (-rx_range_box, rx_range_box))
problem_advise_scale!(pbm, :state, 2, (0, ry_range_box))
problem_advise_scale!(pbm, :state, 3, (-vx_range_box, vx_range_box))
problem_advise_scale!(pbm, :state, 4, (vy_range_box, 0.0))
problem_advise_scale!(pbm, :state, 5, (-θ_range_box, θ_range_box))
problem_advise_scale!(pbm, :state, 6, (-ω_range_box, ω_range_box))
problem_advise_scale!(pbm, :state, 7, mass_range)

problem_advise_scale!(pbm, :input, 1, thrust_range)
problem_advise_scale!(pbm, :input, 2, (-δ_range_box, δ_range_box))

problem_advise_scale!(pbm, :parameter, 1, (0.5*tf_guess, tf_guess));

## Solve the Problem Using PTR

With the trajectory problem fully defined, it's time to select an SCP algorithm to solve it. This comes down to selecting the set of parameters that guide the algorithm's behavior.

The toolbox currently provides three algorithsm: `PTR`, `SCvx`, and `GuSTO`. The latter two are discussed in detail in the [IEEE Control Systems Magazine tutorial paper](https://arxiv.org/abs/2106.09125). The first algorithm (which goes by the full name of "Penalized Trust Region") is described in several papers, most notably an [AIAA Journal of Guidance, Control, and Dynamics paper on rocket landing](https://arxiv.org/abs/1811.10803).

This time we will use PTR to solve the trajectory problem. The parameters below were found to work well. SCP algorithms do not have a generic set of parameters that work well for every problem. Generally, some tuning is required, but afterwards the same set of parameters applies to a wide range of perturbations of the original problem (such as in initial condition, mass, inertia, thrust magnitudes, etc.).

In [181]:
# Parameters
N, Nsub = 20, 20
iter_max = 30
disc_method = FOH
wvc, wtr = 1e1, 1e-1
feas_tol = 5e-3
ε_abs, ε_rel = 1e-5, 1e-4
q_tr = Inf
q_exit = Inf
solver, solver_options = ECOS, Dict("verbose"=>0)      

pars = PTR.Parameters(N, Nsub, iter_max, disc_method, wvc, wtr, ε_abs,
                      ε_rel, feas_tol, q_tr, q_exit, solver, solver_options);

We can then simply "create" the SCP problem, which is a combination of the trajectory problem and the algorithm that solves it.

We subsequently call the `solve` method. This prints a progress table at each iteration of the SCP algorithm.

In [182]:
ptr_pbm = PTR.create(pars, pbm)
sol, history = PTR.solve(ptr_pbm);

k  | status   | vd    | vs    | vbc   | J         | ΔJ %      | Δx    | Δu    | Δp    | δ     | dyn | ηx    | ηu    | ηp   
---+----------+-------+-------+-------+-----------+-----------+-------+-------+-------+-------+-----+-------+-------+------
1  | OPTIMAL  | 6e-10 | 0e+00 | 2e-11 | -8.20e-01 |           | 2e+00 | 5e+00 | 3e-01 | 2e+00 | F   | 1.88  | 5.00  | 0.25 


2  | OPTIMAL  | 3e-10 | 0e+00 | 2e-12 | -8.97e-01 | 9.38      | 4e-01 | 1e+00 | 9e-13 | 4e-01 | F   | 0.38  | 1.00  | 0.00 


3  | OPTIMAL  | 2e-09 | 0e+00 | 1e-11 | -9.37e-01 | 4.52      | 2e-01 | 6e-01 | 2e-11 | 2e-01 | T   | 0.18  | 0.57  | 0.00 


4  | OPTIMAL  | 3e-10 | 0e+00 | 3e-13 | -9.48e-01 | 1.17      | 9e-03 | 3e-02 | 1e-13 | 9e-03 | T   | 0.01  | 0.03  | 0.00 


5  | OPTIMAL  | 2e-08 | 0e+00 | 2e-10 | -9.49e-01 | 0.07      | 6e-05 | 2e-04 | 1e-11 | 6e-05 | T   | 0.00  | 0.00  | 0.00 


6  | OPTIMAL  | 2e-08 | 0e+00 | 2e-10 | -9.49e-01 | 0.00      | 2e-09 | 5e-10 | 2e-11 | 2e-09 | T   | 0.00  | 0.00  | 0.00 


## Plots

In [183]:
xc=[]
_sol = history.subproblems[end].sol

tf = _sol.p[1]
xd = _sol.xd
ud = _sol.ud
vd = _sol.vd
p_sol = _sol.p
td = sol.td*tf

Nc = 100
tc = LinRange(0, tf, Nc)
xc = hcat([sample(sol.xc, t) for t in LinRange(0, 1, Nc)]...)
uc = hcat([sample(sol.uc, t) for t in LinRange(0, 1, Nc)]...)
# tf=hcat([sample(_sol.p[1], t) for t in LinRange(0, 1, Nc)]...)
tf_values = [spbm.sol.p[1] for spbm in history.subproblems]
pushfirst!(tf_values, history.subproblems[1].ref.p[1]);
print(xc)

[499.9999999999857 519.1991784408907 539.4570073776631 560.5568651520648 582.2780793303543 604.4059646703328 626.767391473364 649.2667524288759 671.7958608407405 694.1934374397184 716.238801132451 737.660055015369 758.1655387397958 777.4769609039938 795.3594613626474 811.6130367418699 826.0725848094978 838.6577116874795 849.3400510675856 858.1063263514211 864.9824073503266 869.9951119699218 873.1596327352246 874.4947402261214 874.0073020579315 871.6971098044012 867.5895017427224 861.7184992936375 854.1734427051963 845.0845882207522 834.5750272273817 822.7624896396869 809.7824213596791 795.7890641148688 780.9404749472449 765.3933374596893 749.28613898889 732.739304173146 715.8081032360932 698.4906632807999 680.7860682257777 662.7030799154996 644.2536166257049 625.4609441550413 606.3678696172774 587.0167819959954 567.4495178557794 547.7107481818173 527.8469480542491 507.90988524431623 487.9454506861374 467.9899810901346 448.0714023088208 428.20999356373267 408.42777132136564 388.74970085

 -0.05078604470513851 -0.04991795962561594 -0.04866155072559911 -0.047219383850375454 -0.04584494670244722 -0.04460650457622524 -0.04354980759955019 -0.04271855230384822 -0.04214345648048252 -0.041658889002746724 -0.04120326592452915 -0.04077698478332873 -0.040380614470002865 -0.040011258420178344 -0.03941544982628646 -0.0384759531282651 -0.037273858227180184 -0.035887845147234324 -0.03439646628480805 -0.03287960130807026 -0.0313728564636254 -0.029882625508802045 -0.02841493123938682 -0.02697585359862537 -0.02557532732300092 -0.02423285208522139 -0.022960727145029402 -0.021771119330104994 -0.020675263090997684 -0.01968819398195988 -0.018847626263232695 -0.018175275690371688 -0.01768966555772049 -0.017410596137754385 -0.017323570228020238 -0.016807905245722145 -0.015506866883641 -0.013297814362630438 -0.010054432210076757 -0.005648671812021161 -0.0009444193916662998 0.0032918772773771975 0.007060488287145512 0.010367405735619513 0.013215960583812388 0.015704214520518702 0.01800636098231

In [184]:
x0=xc[:,1]
xplus=[]
dt=0.8286256886085988
N=4940
xc_tmp=zeros(7,N)
# print(tc)
# print(p_sol)
# print(tf_values)
for i=1:N
    # xplus=x0+f(x0,uc[1:2,round(Int,i/100)+1],p_sol)*dt/100
    xplus=x0+f_rocket(x0,uc[1:2,round(Int,i/50)+1])*dt/50
x0=xplus
xc_tmp[1:7,i]=xplus
end
#xc=xc_tmp
# print(round(Int,0.46))
print(xplus)

[21.438434720270898, 69.80929573821541, -2.4390319390163095, 1.0405485290183452, 0.04841822155293783, 0.002990520521118622, 23744.13873660383]

In [185]:
using CSV, DataFrames


ArgumentError: ArgumentError: Package CSV not found in current path.
- Run `import Pkg; Pkg.add("CSV")` to install the CSV package.

In [186]:
using Serialization
# Serialize the data to a file
serialize("tc_data.bin", tc)
serialize("xc_data.bin", xc)
serialize("uc_data.bin", uc)

1600

In [187]:
# Deserialize the data from the files
tc = deserialize("tc_data.bin")
xc = deserialize("xc_data.bin")
uc = deserialize("uc_data.bin")

2×100 Matrix{Float64}:
 70146.3     63124.0       56101.6       …  74447.2        79936.9
    -0.1688     -0.139761     -0.110722         0.0308264      0.0383319

In [188]:
using Plots



In [189]:
# Plot xc vs tc
plot(tc, xc, title="xc vs tc", xlabel="tc", ylabel="xc")

# Plot uc vs tc
plot(tc, uc, title="uc vs tc", xlabel="tc", ylabel="uc")

PyCall.PyError: PyError ($(Expr(:escape, :(ccall(#= /Users/vrushabh/.julia/packages/PyCall/KLzIO/src/pyfncall.jl:43 =# @pysym(:PyObject_Call), PyPtr, (PyPtr, PyPtr, PyPtr), o, pyargsptr, kw))))) <class 'ValueError'>
ValueError('x and y must have same first dimension, but have shapes (100,) and (7, 100)')
  File "/Users/vrushabh/.julia/conda/3/x86_64/lib/python3.10/site-packages/matplotlib/pyplot.py", line 3578, in plot
    return gca().plot(
  File "/Users/vrushabh/.julia/conda/3/x86_64/lib/python3.10/site-packages/matplotlib/axes/_axes.py", line 1721, in plot
    lines = [*self._get_lines(self, *args, data=data, **kwargs)]
  File "/Users/vrushabh/.julia/conda/3/x86_64/lib/python3.10/site-packages/matplotlib/axes/_base.py", line 303, in __call__
    yield from self._plot_args(
  File "/Users/vrushabh/.julia/conda/3/x86_64/lib/python3.10/site-packages/matplotlib/axes/_base.py", line 499, in _plot_args
    raise ValueError(f"x and y must have same first dimension, but "


In [190]:
# Save to CSV files
CSV.write("tc.csv", DataFrame(tc=tc))
CSV.write("xc.csv", DataFrame(xc=xc))
CSV.write("uc.csv", DataFrame(uc=uc))

UndefVarError: UndefVarError: `CSV` not defined

In [191]:
# Load from CSV files
tc = CSV.read("tc.csv", DataFrame)["tc"]
xc = CSV.read("xc.csv", DataFrame)["xc"]
uc = CSV.read("uc.csv", DataFrame)["uc"]

UndefVarError: UndefVarError: `CSV` not defined

In [192]:
include("utils/p4_rocket_graphics.jl");

In [193]:
visualize_trajectory();

BoundsError: BoundsError: attempt to access 2×20 Matrix{Float64} at index [1, 21]

In [194]:
# function plot_trajectory() 

    fig = plt.figure(figsize=(9, 6))

    ctres, overlap = Nc, 3

    # speed
    vct = [norm(xc[3:4, i], 2) for i=1:size(xc[3:4, :])[2]]

    cmap = generate_colormap("viridis"; minval=minimum(vct), maxval=maximum(vct))

    ax = setup_axis!(111, xlabel="Downrange [m]", ylabel="Altitude [m]",
                     axis="equal", cbar=cmap, clabel="Speed [m/s]",
                     cbar_aspect=40)

    ax.grid(color="0.9")

    # Thrust vector profile (inertial frame)
    ux = []
    uy = []
    for i in 1:N
        append!(ux, -ud[1, i].*sin(xd[5, i] + ud[2, i]))
        append!(uy,  ud[1, i].*cos(xd[5, i] + ud[2, i]))
    end

    for i in 1:N
    #     ax.plot(xd[1, i], xd[2, i], linewidth=0, color="darkslategrey", marker=(4, 0, 45+rad2deg(xd[5, i])), ms=5, markerfacecolor="goldenrod", alpha=1, markeredgewidth=1, zorder=100)
        ax.quiver(xd[1, i], xd[2, i], sin(xd[5, i]), -cos(xd[5, i]), color="darkslategrey", pivot="middle", headaxislength=0, headlength=0, antialiased="True", width=0.015, scale=22.5, joinstyle="round", zorder=100)
        ax.quiver(xd[1, i], xd[2, i], -ux[i], -uy[i], color="tomato", pivot="tail", headaxislength=0, headlength=0, antialiased="True", width=0.0035, scale=11.5*T_max, zorder=20)
    end

    ax.plot(xd[1, :], xd[2, :], linewidth=0, color="k", marker="o", ms=5, markerfacecolor="goldenrod", alpha=1, markeredgewidth=0.625, zorder=100000, label="Guidance solution")
    # ax.plot(xc[1, :], xc[2, :], linewidth=1, color="mediumseagreen", label="Continuous-time dynamics", alpha=0.75, zorder=1000)

    span_gs = max(100, 2.25*abs(x_0[1]))
    x_cone = [-span_gs; 0; span_gs]
    ax.plot(x_cone, tan(γ_gs).*abs.(x_cone), linewidth=1.5, linestyle="dashed", color="slategrey", alpha=0.75, dash_capstyle="round", label="Glide-slope")

    line_segs = Vector{Matrix}(undef, 0)
    line_clrs = Vector{NTuple{4, Real}}(undef, 0)
    for k=1:ctres-overlap
        push!(line_segs, xc[1:2, k:k+overlap]')
        push!(line_clrs, cmap.to_rgba(vct[k]))
    end
    trajectory = PyPlot.matplotlib.collections.LineCollection(
        line_segs, zorder=1000, colors = line_clrs, linewidths=2,
        capstyle="round")
    ax.add_collection(trajectory)

    plt.title("\\textbf{Rocket-landing trajectory}", fontsize=13, pad=12.5)
    ax.legend(loc=2)

    fig.savefig("traj.png", format="png", bbox_inches = "tight")
    ;
    
# end

# plot_trajectory();

BoundsError: BoundsError: attempt to access 2×20 Matrix{Float64} at index [1, 21]

In [195]:
# function plot_states()
    
    fig, axs = plt.subplots(3, 2, sharex="col", figsize=(12, 8))

    line1 = axs[1, 1].plot(tc, xc[1, :], linewidth=1.5, color="darkslategrey", label="Propagated")
    line2 = axs[1, 1].plot(td, xd[1, :], linewidth=0, color="k", marker="o", ms=5, markerfacecolor="mediumspringgreen", alpha=0.75, label="Guidance", clip_on=false, zorder=1e8)
    axs[1, 1].set_ylabel("Downrange [m]", fontsize=12)
    axs[1, 1].tick_params(bottom=false)
    axs[1, 1].grid(color="0.9")
    # axs[1, 1].legend(fontsize=8)

    line1 = axs[2, 1].plot(tc, xc[2, :], linewidth=1.5, color="darkslategrey", label="Propagated")
    line2 = axs[2, 1].plot(td, xd[2, :], linewidth=0, color="k", marker="o", ms=5, markerfacecolor="orangered", alpha=0.75, label="Guidance", clip_on=false, zorder=1e8)
    axs[2, 1].set_ylabel("Altitude [m]", fontsize=12)
    axs[2, 1].tick_params(bottom=false)
    axs[2, 1].grid(color="0.9")
    # axs[2, 1].legend(fontsize=8)

    line1 = axs[3, 1].plot(tc, [rad2deg(xc[5, i]) for i in 1:Nc], linewidth=1.5, color="darkslategrey", label="Propagated")
    line2 = axs[3, 1].plot(td, [rad2deg(xd[5, i]) for i in 1:N], linewidth=0, color="k", marker="o", ms=5, markerfacecolor="dodgerblue", alpha=0.75, label="Guidance", clip_on=false, zorder=1e8)
    axs[3, 1].set_ylabel("Pitch [\$^\\circ\$]", fontsize=12)
    axs[3, 1].grid(color="0.9")
    # plt.legend(fontsize=8)

    line1 = axs[1, 2].plot(tc, xc[3, :], linewidth=1.5, color="darkslategrey", label="Propagated")
    line2 = axs[1, 2].plot(td, xd[3, :], linewidth=0, color="k", marker="o", ms=5, markerfacecolor="mediumspringgreen", alpha=0.75, label="Guidance", clip_on=false, zorder=1e8)
    axs[1, 2].set_ylabel("Longitudinal velocity [m/s]", fontsize=12)
    axs[1, 2].tick_params(bottom=false)
    axs[1, 2].grid(color="0.9")
    # axs[1, 2].legend(fontsize=8)

    line1 = axs[2, 2].plot(tc, xc[4, :], linewidth=1.5, color="darkslategrey", label="Propagated")
    line2 = axs[2, 2].plot(td, xd[4, :], linewidth=0, color="k", marker="o", ms=5, markerfacecolor="orangered", alpha=0.75, label="Guidance", clip_on=false, zorder=1e8)
    axs[2, 2].set_ylabel("Rate-of-descent [m/s]", fontsize=12)
    axs[2, 2].tick_params(bottom=false)
    axs[2, 2].grid(color="0.9")
    # axs[2, 2].legend(fontsize=8)

    line1 = axs[3, 2].plot(tc, [rad2deg(xc[6, i]) for i in 1:Nc], linewidth=1.5, color="darkslategrey", label="Propagated")
    line2 = axs[3, 2].plot(td, [rad2deg(xd[6, i]) for i in 1:N], linewidth=0, color="k", marker="o", ms=5, markerfacecolor="dodgerblue", alpha=0.75, label="Guidance", clip_on=false, zorder=1e8)
    axs[3, 2].set_ylabel("Pitch rate [\$^\\circ\$/s]", fontsize=12)
    axs[3, 2].grid(color="0.9")
    # plt.legend(fontsize=8)

    # handles, labels = axs[2, 1].get_legend_handles_labels()
    # fig.legend(handles, labels, loc="upper center")

    legend_elements = [PyPlot.matplotlib.lines.Line2D([0], [0], linewidth=0, color="k", marker="o", ms=5, markerfacecolor="white", alpha=0.75, label="Guidance", clip_on=false, zorder=1e8),
                       PyPlot.matplotlib.lines.Line2D([0], [0], color="darkslategrey", linewidth=1.5, label="Propagated"),]

    fig.legend(handles=legend_elements, ncol=2, loc="lower right", bbox_to_anchor=(0, 0.9075, 0.9075, 0), fontsize="12", mode="normal")

    # fig.legend(handles=legend_elements, loc="upper center")

    axs[3, 1].set_xlabel("Time [s]", fontsize=12)
    axs[3, 2].set_xlabel("Time [s]", fontsize=12)
    fig.savefig("states.pdf", format="pdf", bbox_inches = "tight")

    ;
    
# end
# plot_states();

BoundsError: BoundsError: attempt to access 7×20 Matrix{Float64} at index [5, 21]

In [196]:
plot_mass();

In [197]:
# function plot_controls()
    
    #########################
    ### Thrust magnitude ####
    #########################
    
    fig = plt.figure(figsize=(6, 3))
    ax = fig.add_subplot(111)

    ax.grid(color="0.9")

    ax.plot(td, ud[1, :]/1e3, linewidth=1.5, color="darkslategrey", marker="o", ms=5, markeredgecolor="k", markerfacecolor="goldenrod", alpha=1, clip_on=false, zorder=1e8)
    ax.axhline(y=T_min/1e3, color="steelblue", linewidth=1.5, linestyle="dashed", dash_capstyle="round", label="Throttle limits")
    ax.axhline(y=T_max/1e3, color="steelblue", linewidth=1.5, linestyle="dashed", dash_capstyle="round")

    ax.legend(loc="best", bbox_to_anchor=(0.5, 0.05, 0.5, 0.5))

    ax.set_xlabel("Time [s]")
    ax.set_ylabel("Thrust magnitude [kN]");
    
    #####################
    ### Gimbal angle ####
    #####################
    
    fig = plt.figure(figsize=(6, 3))
    ax = fig.add_subplot(111)

    ax.grid(color="0.9")

    ax.plot(td, [rad2deg(ud[2, i]) for i in 1:N], linewidth=1.5, color="darkslategrey", marker="o", ms=5, markeredgecolor="k", markerfacecolor="mediumspringgreen", alpha=1, clip_on=false, zorder=1e8)
    ax.axhline(y=rad2deg(δ_max), color="tomato", linewidth=1.5, linestyle="dashed", dash_capstyle="round", label="Gimbal limits")
    ax.axhline(y=-rad2deg(δ_max), color="tomato", linewidth=1.5, linestyle="dashed", dash_capstyle="round")

    ax.legend(loc="best", bbox_to_anchor=(0.5, 0.05, 0.5, 0.5))

    ax.set_xlabel("Time [s]")
    ax.set_ylabel("Gimbal angle [\$^\\circ\$]")
    fig.savefig("controls.pdf", format="pdf", bbox_inches = "tight")

    ;
    
# end
# plot_controls();

BoundsError: BoundsError: attempt to access 2×20 Matrix{Float64} at index [2, 21]

In [198]:
plot_ToF();